In [1]:
path_to_file = keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)
print(path_to_file)

with open(path_to_file) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

shakespeare_tensor = tf.constant([shakespeare_text])

NameError: name 'keras' is not defined

In [ ]:
vocab = list(set(shakespeare_text.lower().strip()))
print(vocab)
print(f"vocab size = {len(vocab)}")

['m', '&', '\n', 'r', 'a', '?', ' ', 'y', ':', 't', 's', 'p', 'd', 'f', 'q', 'x', 'b', '-', 'k', '$', 'z', ',', 'j', 'e', 'w', 'u', 'c', 'n', 'l', "'", ';', '!', 'v', 'g', 'o', '3', '.', 'i', 'h']
vocab size = 39


In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(shakespeare_text.lower())
config = tokenizer.get_config()
tokenizer.texts_to_sequences(["hello world", "hello world"])

[[7, 2, 12, 12, 4, 1, 17, 4, 9, 12, 13],
 [7, 2, 12, 12, 4, 1, 17, 4, 9, 12, 13]]

In [ ]:
n_tokens = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
print(f"{encoded[:100]}")

[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
  0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
  4  8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24
 17  0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23
 10 15  3 13]


# Create Dataset

In [ ]:
train_size = dataset_size * 90 // 100
print(f"train_size = {train_size}")

train_size = 1003854


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded)
#dataset = dataset.repeat()
n_steps = 50
dataset = dataset.window(n_steps + 1, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window_ds: window_ds.batch(n_steps + 1))

In [ ]:
for x in dataset.take(2):
    print(x)

tf.Tensor(
[19  5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1
  0 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1
  4  8  0], shape=(51,), dtype=int64)
tf.Tensor(
[ 5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0
 22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4
  8  0 14], shape=(51,), dtype=int64)


In [ ]:
dataset = dataset.batch(128)

In [ ]:
for x in dataset.take(1):
    print(x.shape)

(128, 51)


In [ ]:
dataset = dataset.map(lambda window: (window[:, :-1], window[:, 1:]))

In [ ]:
dataset = dataset.cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for x, y in dataset.take(1):
    print(f"x = {x.shape}")
    print(f"y = {y.shape}")    

x = (128, 50)
y = (128, 50)


2025-06-05 21:31:32.969684: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
transform = lambda x, y: (tf.one_hot(x, depth=n_tokens), y)
dataset = dataset.map(transform)
for x, y in dataset.take(1):
    print(f"x = {x.shape}")
    print(f"y = {y.shape}")

x = (128, 50, 39)
y = (128, 50)


2025-06-05 21:31:33.052670: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-06-05 21:31:33.193007: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Create Models

In [ ]:
m1 = keras.Sequential(
    [
        keras.layers.Input(shape=[None, n_tokens]),
        keras.layers.GRU(128, return_sequences=True),
        keras.layers.Dense(n_tokens, activation="softmax")
    ]
)
m1.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m1.summary()
model = m1

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_1 (GRU)                     │ (None, None, 128)      │        64,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,927 (273.15 KB)

 Trainable params: 69,927 (273.15 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
text = "to be "
tokens = tokenizer.texts_to_sequences([text])
inp_tf = tf.constant(tokens)
inp_tf = tf.one_hot(inp_tf, depth=n_tokens)
print(inp_tf.shape)
y = model.predict(inp_tf)
print(y.shape)
y = tf.argmax(y[0,-1])

(1, 6, 39)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
(1, 6, 39)


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
2.19.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)
history = m1.fit(dataset, epochs=10, steps_per_epoch=train_size // 128, callbacks=[early_stopping], verbose=2)
m1.save("models/shakespeare_gru_1.keras")

Epoch 1/10
7842/7842 - 30s - 4ms/step - loss: 1.9055
Epoch 2/10
7842/7842 - 3s - 433us/step - loss: 1.6977
Epoch 3/10


2025-06-05 21:32:07.233299: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 6788505307422867365
2025-06-05 21:32:07.233320: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 11210115410648602823
2025-06-05 21:32:07.233326: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10434652009565501561
2025-06-05 21:32:07.233329: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 7603395054250313659
2025-06-05 21:32:07.233334: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16828180969127122854
/opt/conda/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may

7842/7842 - 36s - 5ms/step - loss: 1.6178
Epoch 4/10
7842/7842 - 4s - 468us/step - loss: 1.5901
Epoch 5/10


2025-06-05 21:32:47.023113: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10434652009565501561
2025-06-05 21:32:47.023138: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10363305953241996774
2025-06-05 21:32:47.023147: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16828180969127122854


7842/7842 - 40s - 5ms/step - loss: 1.5415
Epoch 6/10
7842/7842 - 4s - 558us/step - loss: 1.5402
Epoch 7/10


2025-06-05 21:33:31.668118: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10434652009565501561
2025-06-05 21:33:31.668130: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 16828180969127122854


7842/7842 - 40s - 5ms/step - loss: 1.5020
Epoch 8/10
7842/7842 - 5s - 617us/step - loss: 1.5115
Epoch 9/10


2025-06-05 21:34:16.925301: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 6788505307422867365
2025-06-05 21:34:16.925316: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 11210115410648602823
2025-06-05 21:34:16.925320: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_8]]
2025-06-05 21:34:16.925332: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10434652009565501561
2025-06-05 21:34:16.925334: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 7603395054250313659


7842/7842 - 40s - 5ms/step - loss: 1.4775
Epoch 10/10
7842/7842 - 4s - 503us/step - loss: 1.4921


2025-06-05 21:35:00.719614: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 6788505307422867365
2025-06-05 21:35:00.719633: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 11210115410648602823
2025-06-05 21:35:00.719643: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10434652009565501561
2025-06-05 21:35:00.719648: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 7603395054250313659


In [ ]:
m2 = keras.Sequential(
    [
        keras.layers.Input(shape=[None, n_tokens]),
        keras.layers.LSTM(128, return_sequences=True),
        keras.layers.Dense(n_tokens, activation="softmax")
    ]
)
m2.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m2.summary()
early_stopping = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)
# start_time = time.time.now()
h2 = m2.fit(dataset, epochs=10, steps_per_epoch=train_size // 128, callbacks=[early_stopping], verbose=2)
# runtime = time.perf_counter() - start_time
# print(f"Time {runtime:,.0f} seconds.")
m2.save("models/shakespeare_lstm_1.keras")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, None, 128)      │        86,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,047 (355.65 KB)

 Trainable params: 91,047 (355.65 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
m3 = keras.Sequential(
        [
                keras.layers.Input(shape=(None, n_tokens)),
                keras.layers.GRU(128, return_sequences=True),
                keras.layers.GRU(128, return_sequences=True),
                keras.layers.Dense(n_tokens, activation="softmax")
                ],
        )
m3.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
m3.summary()
model = m3

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.002, patience=2)
start_time = datetime.now()
history3 = m3.fit(dataset, epochs=10, steps_per_epoch=train_size // 128, callbacks=[early_stopping], verbose=2)
elapsed_time3 = datetime.now() - start_time
print(f"Training completed in {elapsed_time3.total_seconds():.0f} seconds.")
m3.save("models/shakespeare_gru_2.keras")

In [ ]:
import matplotlib.pyplot as plt

# Extract the loss values from the three history objects
loss1 = history.history['loss']
loss2 = h2.history['loss']
loss3 = history3.history['loss']

# Plot the training losses
plt.plot(loss1, label="Run 1 (GRU)", linestyle='-', color='blue')
plt.plot(loss2, label="Run 2 (LSTM)", linestyle='--', color='orange')
plt.plot(loss3, label="Run 3 (Stacked GRU)", linestyle='-.', color='green')

# Add labels, title, and legend
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Comparison of Training Histories")
plt.legend()

# Display the plot
plt.show()